In [12]:
#import os
import pandas as pd
import os
import numpy as np
import numpy.ma as ma 
import awkward as ak
from tqdm import tqdm
import torch
import torch_geometric as tg
import torch_geometric
from torch_geometric.data import Data
#import torch
from torch_geometric.data import InMemoryDataset, download_url
import torch_geometric.transforms as T

#NOTE: NEW 2/20/23 
from typing import List, Union

from torch_geometric.data import Data, HeteroData
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform 

torch.cuda.empty_cache()
root = '/work/clas12/users/mencke/pyg_test_rec_traj_dataset_111'
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.load(self.processed_paths[0])
        # For PyG<2.4:
        # self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        # Read data into huge `Data` list.
        data_list = None

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        self.save(data_list, self.processed_paths[0])
        # For PyG<2.4:
        # torch.save(self.collate(data_list), self.processed_paths[0])

# Create PyG Dataset
#root = '/work/clas12/users/mfmce/pyg_test_rec_traj_dataset_5_28_24/' # 3_14_24 #OLD
dataset = MyOwnDataset(
            root,
            transform=None, #T.Compose([T.ToUndirected(),T.KNNGraph(k=6)]),
            pre_transform=None,
            pre_filter=None
        )
dataset

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GraphConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.norm import GraphNorm, BatchNorm 

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
#         torch.manual_seed(12345)
        self.conv1 = GCNConv(in_channels, hidden_channels)#.jittable() #NOTE: NEEDED FOR DEPLOYMENT IN CMAKE
        self.conv2 = GCNConv(hidden_channels, hidden_channels)#.jittable()
        #self.block2 = nn.DataParallel(self.block2)
        #self.conv2 = torch.nn.DataParallel(self.conv2) #this was trying the parallization thing. 
        self.conv3 = GCNConv(hidden_channels, hidden_channels)#.jittable()
        #self.conv3 = torch.nn.DataParallel(self.conv3)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, out_channels)
        self.bn1 = torch_geometric.nn.norm.GraphNorm(hidden_channels)
        self.bn2 = torch_geometric.nn.norm.GraphNorm(hidden_channels)
        self.bn3 = torch_geometric.nn.norm.GraphNorm(hidden_channels)

    def forward(self, x, edge_index, batch): 
        # 1. Obtain node embeddings 
#         print("x = ",x)
#         print("DEBUGGING: in GCN: begin: x.requires_grad = ",x.requires_grad)
        x = self.conv1(x, edge_index) #input layer
#         print("DEBUGGING: in GCN: self.conv1(x, edge_index): x.requires_grad = ",x.requires_grad)
        x = self.bn1(x) #normalize it
#         print("DEBUGGING: in GCN: self.bn2(x): x.requires_grad = ",x.requires_grad)
#         print("self.conv1(x,edge_index) = ",x)
        x = x.relu() #activation
#         x = torch.nn.function.elu(x)
#         print("DEBUGGING: in GCN: x.relu(): x.requires_grad = ",x.requires_grad)
#         print("x.relu() = ",x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
#         print("DEBUGGING: in GCN: self.bn2(x): x.requires_grad = ",x.requires_grad)
#         print("self.conv2(x,edge_index) = ",x)
        x = x.relu() 
#         print("x.relu() = ",x)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
#         print("self.conv3(x,edge_index) = ",x) 
#         print("DEBUGGING: in GCN: self.bn3(x): x.requires_grad = ",x.requires_grad)

#         # 2. Readout layer
        x = global_mean_pool(x, batch) #what is this for.           # [batch_size, hidden_channels]
#         print("self.conv2(global_mean_pool(x, batch)) = ",x)
#         print("DEBUGGING: in GCN: global_mean_pool(x): x.requires_grad = ",x.requires_grad)

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training) #for overfitting
#         print("DEBUGGING: in GCN: F.dropout: x.requires_grad = ",x.requires_grad)
#         print("F.dropout(x, p=0.5, training=self.training) = ",x)
        x = self.lin1(x) #why threee output layers thigns 
        x = self.lin2(x)
        x = self.lin3(x)
#         print("DEBUGGING: in GCN: self.lin*(x): x.requires_grad = ",x.requires_grad)
#         print("self.lin3(x) = ",x)
#         x = torch.sigmoid(x) #NOTE: DON'T SOFTMAX IF USING BCELOSS, USE SIGMOID INSTEAD
#         print("torch.sigmoid(x) = ",x)
#         print("DEBUGGING: in GCN: torch.sigmoid(x): x.requires_grad = ",x.requires_grad)
        return x

model = GCN(dataset.num_node_features,64,2)
# model = GIN(in_channels=dataset.num_node_features,out_channels=2)
#print(model)
#print("\ndataset[0].pos = ",dataset[0].pos)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
devicee = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu') 
print("Device = ",device)
model = model.to(device)
print("DEBUGGING: torch.cuda.is_available() = ",torch.cuda.is_available())

from torch.utils.data import random_split #TODO: SEE IF YOU CAN USE THIS
# torch.manual_seed(12345)
# print('DEBUGGING: BEFORE: dataset.y.shape = ',dataset.y.shape)
dataset = dataset.shuffle() #shuffle (randmoize placement of it) not sure if this is needed. 
#print('DEBUGGING: AFTER:  dataset.y.shape = ',dataset.y.shape)

#print(len(dataset)) 

fracs = [0.8, 0.1, 0.1] #percent of dataset used for training testing and validatoin 80%,10%,10% #NOTE: SHOULD CHECK np.sum(fracs) == 1 and len(fracs)==3
fracs = [torch.sum(torch.tensor(fracs[:idx])) for idx in range(1,len(fracs)+1)] #get the indexes for training ... parts to use. 
#print(fracs)
split1, split2 = [int(len(dataset)*frac) for frac in fracs[:-1]] 
train_dataset = dataset[:split1]
val_dataset = dataset[split1:split2]
test_dataset = dataset[split2:] 

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of validation graphs: {len(val_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}') 

from torch_geometric.loader import DataLoader 
#from torch.utils.data import WeightedRandomSampler

batch_size = 18
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)#, drop_last=True)
val_loader = DataLoader(val_dataset,  batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) 

model = GCN(dataset.num_node_features, dataset.num_classes, 2).to(device) #initiate the model, #2 is the number of outputs here is 2 as pion_z, proton_z 

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) #get the optimizer

data_labels = train_dataset.y 
#weight_signal = counts[1]/counts[0]#DEBUGGING MULTIPLY BY 2 ...
#print("weight_signal = ",weight_signal) 
# weight = torch.FloatTensor([weight_signal, 1.0]).to(device) #NOTE: That labels are [sg?,bg?] so label 0 in this case is sg and label 1 is bg.
losss = torch.nn.MSELoss(reduction = 'mean').to(device)


def train():
    model.train() #initailize the model
    #for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg
    for i,data in enumerate(train_loader):
        data = data.to(device) #switch to GPU 
        optimizer.zero_grad() #
        out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass
        yy = []
        for j in range(0,2*len(out),2): 
            fuckk = data.y[j]; fuckj = data.y[j+1]
            yy +=[[fuckk.item(),fuckj.item()]]
        yy = torch.tensor(yy).to(device)
        loss = losss(out, yy).to(device) #compute the loss
        loss.backward() #get the gradients. 
        optimizer.step() #take a step. 
        
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score 

def test(loader):
    length = len(loader.dataset)
    model.eval() #evaluate teh model.
    
    #mse_tot = []
    mse_total = 0
    mse_pi = 0
    mse_p = 0
    #r
    #for data in tqdm(loader):  # Iterate in batches over the training/test dataset.
    for data in loader:
        data = data.to(device) #put to GPU 
        out = model(data.x, data.edge_index, data.batch).to(device) #evalueate 
        #this and the for loop is converting data.y to a tensor in the same shape as out rows and 2 columns first is y_pion second is y_proton
        yy = []
        for j in range(0,2*len(out),2): 
            yy +=[[data.y[j].item(),data.y[j+1].item()]]
        yy = torch.tensor(yy).to(device)
        loss = losss(out, yy).cpu() #getting teh loss function
        mse_total+=loss.item() #getting the mse (total) 
        for j in range(len(out)): 
            #x_pi = out[j][0]; x_p =out[j][1]
            
            mse_pi += (out[j][0].item()-yy[j][0].item() )**2
            mse_p +=(out[j][1].item()-yy[j][1].item())**2
            
        #)
    return mse_total/length, np.sqrt(mse_total/length), mse_pi/length, np.sqrt(mse_pi/length), mse_p/length, np.sqrt(mse_p/length)

        
        
def print_out():
    model.eval() #initailize the model
    #for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg
    outt= []
    for i,data in enumerate(train_loader):
        data = data.to(devicee) #switch to GPU 
        optimizer.zero_grad() #
        out = model(data.x, data.edge_index, data.batch).to(devicee)  # Perform a single forward pass
        #yy = []
        #for j in range(0,2*len(out),2):
        #    fuckk = data.y[j]; fuckj = data.y[j+1]
        #    yy +=[[fuckk.item(),fuckj.item()]]
        #yy = torch.tensor(yy).to(device)
        #loss = losss(out, yy).cpu() #compute the loss
        outt+=[out.cpu()]
    return outt 


# for name, param in model.named_parameters(): 

Device =  cuda:0
DEBUGGING: torch.cuda.is_available() =  True
Number of training graphs: 7231
Number of validation graphs: 904
Number of test graphs: 904


In [13]:
nepochs =  2
train_metrics = {'mse':[], "rmse":[], 'mse_pi':[], 'rmse_pi':[], 'mse_p':[], 'rmse_p':[] }
vall_metrics = {'mse':[], "rmse":[], 'mse_pi':[], 'rmse_pi':[], 'mse_p':[], 'rmse_p':[] }

for epoch in range(nepochs):
    #print("BEFORE TRAIN()") 
    train()
    #print("BEFORE TEST(TRAIN_LOADER)")
    train_mse, train_rmse, train_mse_pi, train_rmse_pi, train_mse_p, train_rmse_p = test(train_loader)
    
    train_metrics['mse'].append(train_mse)
    train_metrics['rmse'].append(train_rmse)
    train_metrics['mse_pi'].append(train_mse_pi)
    train_metrics['rmse_pi'].append(train_rmse_pi) 
    train_metrics['mse_p'].append(train_mse_p)
    train_metrics['rmse_p'].append(train_rmse_p)
    
    #print("BEFORE TEST(VAL_LOADER)")
    vall_mse, vall_rmse, vall_mse_pi, vall_rmse_pi, vall_mse_p, vall_rmse_p = test(val_loader) 
    #if epoch==0 or val_roc_auc >np.max(val_metrics["roc_auc"]) :
    #    model_best_auc = model
    #    PATH = '/work/clas12/users/mfmce/CLAS12_Lambda_resolution_REU_2023/model_best_auc.pt'
    #    torch.save({
    #        'epoch': epoch,
    #        'model_state_dict': model.state_dict(),
    #        'optimizer_state_dict': optimizer.state_dict(),
 #             'loss': loss,
    #        }, PATH) 
    
    vall_metrics['mse'].append(vall_mse)
    vall_metrics['rmse'].append(vall_rmse)
    vall_metrics['mse_pi'].append(vall_mse)
    vall_metrics['rmse_pi'].append(vall_mse)
    vall_metrics['mse_p'].append(vall_mse)
    vall_metrics['rmse_p'].append(vall_mse)  
    if epoch==0:
        print("Epoch ",epoch," Train mse: ",train_mse," Train rmse: ",train_rmse," Train mse pion: ",train_mse_pi,
              " Train rmse pion: ",train_rmse_pi, "Train mse proton:",train_mse_p, "Train rmse proton:",train_rmse_p)
        print("Epoch ",epoch," Validation mse: ",vall_mse," Validation rmse: ",vall_rmse," Validation mse pion: ",vall_mse_pi,
              " Validation rmse pion: ",vall_rmse_pi, "Validation mse proton:",vall_mse_p, "Validation rmse proton:", vall_rmse_p) 
    if epoch%9==0: 
        print("Epoch ",epoch," Train mse: ",train_mse," Train rmse: ",train_rmse," Train mse pion: ",train_mse_pi,
              " Train rmse pion: ",train_rmse_pi, "Train mse proton:",train_mse_p, "Train rmse proton:",train_rmse_p)
        print("Epoch ",epoch," Validation mse: ",vall_mse," Validation rmse: ",vall_rmse," Validation mse pion: ",vall_mse_pi,
              " Validation rmse pion: ",vall_rmse_pi, "Validation mse proton:",vall_mse_p, "Validation rmse proton:", vall_rmse_p) 
    if epoch==1:
        #a = print_out() 
        print("Epoch ",epoch," Train mse: ",train_mse," Train rmse: ",train_rmse," Train mse pion: ",train_mse_pi,
              " Train rmse pion: ",train_rmse_pi, "Train mse proton:",train_mse_p, "Train rmse proton:",train_rmse_p)
        print("Epoch ",epoch," Validation mse: ",vall_mse," Validation rmse: ",vall_rmse," Validation mse pion: ",vall_mse_pi,
              " Validation rmse pion: ",vall_rmse_pi, "Validation mse proton:",vall_mse_p, "Validation rmse proton:", vall_rmse_p) 
model.eval() #initailize the model
    #for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg
outt= []
for i,data in enumerate(train_loader):
    data = data.to(devicee) #switch to GPU 
    optimizer.zero_grad() #
    out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass
    #yy = []
    #for j in range(0,2*len(out),2):
    #    fuckk = data.y[j]; fuckj = data.y[j+1]
    #    yy +=[[fuckk.item(),fuckj.item()]]
    #yy = torch.tensor(yy).to(device)
    #loss = losss(out, yy).cpu() #compute the loss
    outt+=[out]
#return outt  0

IndexError: Caught IndexError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/w/hallb-scshelf2102/clas12/users/mencke/venv/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/w/hallb-scshelf2102/clas12/users/mencke/venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/w/hallb-scshelf2102/clas12/users/mencke/venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/w/hallb-scshelf2102/clas12/users/mencke/venv/lib/python3.8/site-packages/torch_geometric/nn/conv/gcn_conv.py", line 241, in forward
    edge_index, edge_weight = gcn_norm(  # yapf: disable
  File "/w/hallb-scshelf2102/clas12/users/mencke/venv/lib/python3.8/site-packages/torch_geometric/nn/conv/gcn_conv.py", line 99, in gcn_norm
    edge_index, ed

In [ ]:
/work/clas12/users/mencke/venv/slurmm.py

In [ ]:
import matplotlib.pyplot as plt

epochs = [i for i in range(len(train_metrics["mse"]))] 
plt.figure()
plt.title('Training epoch vs. MSE') 
plt.plot(epochs, train_metrics['mse'])
plt.xlabel('epoch')
plt.ylabel('MSE')


plt.title('Validation epoch vs. MSE') 
plt.plot(epochs, vall_metrics['mse'])
plt.xlabel('epoch') 
plt.ylabel('MSE')
print(train_metrics['mse'])
print(vall_metrics['mse']) 

In [ ]:
sbatch submit.sh >> ouiyfgh.txt 
seff 23308894 (job number)

Apptainer> squeue -u mencke
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          23305374   jupyter jupyterh   mencke  R    2:26:05      1 sciml2102
Apptainer> ls /farm_out/mencke
clas12Lambdastuff-23308800-sciml2401.err    jupyterhub-spawner-22293434-farm1942.log    jupyterhub-spawner-22758730-farm180246.log  jupyterhub-spawner-23274871-sciml2101.log
clas12Lambdastuff-23308800-sciml2401.out    jupyterhub-spawner-22456262-farm180247.log  jupyterhub-spawner-22877740-sciml2101.log   jupyterhub-spawner-23305374-sciml2102.log
jupyterhub-spawner-21998791-farm180246.log  jupyterhub-spawner-22559614-sciml2102.log   jupyterhub-spawner-23013279-farm180246.log
jupyterhub-spawner-22222528-farm2325.log    jupyterhub-spawner-22668376-farm180246.log  jupyterhub-spawner-23128719-sciml2402.log
Apptainer> clas12Lambdastuff-23308800-sciml2401.out
bash: clas12Lambdastuff-23308800-sciml2401.out: command not found
Apptainer> emacs clas12Lambdastuff-23308800-sciml2401.out
Apptainer> emacs /farm_out/mencke/clas12Lambdastuff-23308800-sciml2401.out
Apptainer> emacs /farm_out/mencke/clas12Lambdastuff-23308800-sciml2401.err
Apptainer> echo idk >> testtttt_file.txt
Apptainer> cat testtttt_file.txt 
idk
Apptainer> sbatch submit.sh >> ouiyfgh.txt
Apptainer> cat ouiyfgh.txt 
Submitted batch job 23308894
Apptainer> seff 23308894
Job ID: 23308894
Cluster: scicomp
User/Group: mencke/root
State: COMPLETED (exit code 0)
Cores: 1
CPU Utilized: 00:00:00
CPU Efficiency: 0.00% of 00:00:03 core-walltime
Job Wall-clock time: 00:00:03
Memory Utilized: 1.38 MB
Memory Efficiency: 0.03% of 3.91 GB
Apptainer> emacs submit.sh
Apptainer> emacs job.sh

In [ ]:
#another examplee

Apptainer> cd /work/clas12/users/mencke
Apptainer> ls
dir_name_2    job.sh   ouiyfgh.txt                   pyg_test_rec_traj_dataset_111  pyg_test_rec_traj_dataset_5_28_24  pyg_test_rec_traj_dataset_69_99  submit.sh   testtttt_file.txt
iiiiiiii.txt  job.sh~  pyg_test_rec_traj_dataset_11  pyg_test_rec_traj_dataset_22   pyg_test_rec_traj_dataset_69_9     pyg_test_rec_traj_dataset_69_9p  submit.sh~  venv
Apptainer> /dir_name_2
bash: /dir_name_2: No such file or directory
Apptainer> cd /work/clas12/users/mencke/dir_name_2
Apptainer> ls
testt_one  testt.py
Apptainer> cd /work/clas12/users/mencke/dir_name_2
Apptainer> cd /work/clas12/users/mencke/dir_name_2
Apptainer> ls
testt_one  testt_one.py  testt.py
Apptainer> emacs submit.sh
Apptainer> cd /work/clas12/users/mencke
Apptainer> cd /work/clas12/users/mencke
Apptainer> emacs submit.sh
Apptainer> emacs job.sh
Apptainer> emacs job.sh
Apptainer> sbatch submit.sh
Submitted batch job 23321341

In [ ]:
numpy
uproot
10:14
awkward
10:15
pandas
matplotlib
10:15
hipopy